In [ ]:
# !pip install openai
# !rm /work/pi_dhruveshpate_umass_edu/project_19/aishwarya/696DS-named-entity-extraction-and-linking-for-KG-construction/code/openai/credentials.json

In [8]:
import os
import csv
import time
import json
import pandas as pd
from tqdm import tqdm

from openai import OpenAI
from dotenv import load_dotenv

In [9]:
load_dotenv("credentials.env")
api_key = os.getenv("api_key")

In [10]:
client = OpenAI(api_key=api_key)
model = "gpt-4" # gpt-3.5-turbo-instruct gpt-4

## Get Data

In [11]:
data_path = "/work/pi_dhruveshpate_umass_edu/project_19/aishwarya/696DS-named-entity-extraction-and-linking-for-KG-construction/"
file = "datasets/synthetic_data_Apr25/SD_Apr25_chunked_data.json"

with open(os.path.join(data_path, file), 'r') as f:
    data = json.load(f)

In [7]:
# type(data), data.keys()

### RElations

In [25]:
# entities_data = pd.read_csv(os.path.join(data_path, "datasets/synthetic_data_Apr25/news_training_processed.csv"))
entities_data = pd.read_csv("/work/pi_dhruveshpate_umass_edu/project_19/aishwarya/696DS-named-entity-extraction-and-linking-for-KG-construction/datasets/synthetic_data_Apr25/news_training_processed.csv")
entities_data = entities_data.rename(columns={"Unnamed: 0": 'doc_key'})

print(entities_data.shape)
entities_data.head()

(924, 8)


,doc_key,id,article,entity types,relationships,synthetic article,processed_entities,processed_relations
0,0,62b1e91ce87f1fc6c056ce9172f5f7c66efba907,By . Lydia Warren . Jerry Sandusky's adopted s...,"[['Jerry Sandusky', 'Person'], ['Matthew Sandu...","[['Matthew Sandusky', 'gave an interview to', ...",In a revealing and emotional interview with Op...,Jerry Sandusky; Matthew Sandusky; Oprah Winfre...,Matthew Sandusky | gave an interview to | Opra...
1,1,ff693482cdd3dfb9e7064f487516b58f20c060ca,"By . Jaymi Mccann . PUBLISHED: . 15:44 EST, 14...","[['Jaymi McCann', 'Person'], ['swimmer', 'Pers...","[['emergency services', 'conduct search and re...","In the scenic Brecon Beacons of Mid Wales, a s...",Jaymi McCann; swimmer; emergency services; pol...,emergency services | conduct search and rescue...
2,2,e9900c3b37b1ca3c0a62d203670f7e0e5ac5da89,By . Michael Zennie . A sightseer who tried to...,"[['Michael Zennie', 'Person'], ['sightseer', '...","[['Grand Prismatic Spring', 'contains', 'bacte...",In the heart of Wyoming lies the Yellowstone N...,Michael Zennie; sightseer; Yellowstone; Grand ...,Grand Prismatic Spring | contains | bacteria\n...
3,3,fe9d8a82cb4122fdd905012becc7418e2d2ff740,Facebook has started collecting information ab...,"[['Facebook', 'Organization'], ['users', 'Conc...","[['Facebook', 'collects information from', 'us...",In a bold move that has stirred up the digital...,Facebook; users; social media site; privacy po...,Facebook | collects information from | users\n...
4,4,aceee806bdda590589d85f67401e6a6a2fc46b2f,"By . Deborah Arthurs . PUBLISHED: . 08:32 EST,...","[['Duchess of Cambridge', 'Person'], ['Hobbs',...","[['Duchess of Cambridge', 'wore clothing from'...",In a remarkable display of support for the Bri...,Duchess of Cambridge; Hobbs; Celeste coat; Liv...,Duchess of Cambridge | wore clothing from | Ho...


In [26]:
entities_data["processed_entities"][0]

'Jerry Sandusky; Matthew Sandusky; Oprah Winfrey; OWN; Penn State; Dottie Sandusky; NBC; Sundance Film Festival; Pennsylvania; Tom Corbett; Victim 1'

In [6]:
entities_data.loc[lambda df: df.doc_key == 3]

,doc_key,chunk_index,entities,latency,input_tokens,output_tokens,processed_entities
32,3,0,All News Today; 2012/04/09; ABILA; Kronos; Sil...,1.462747,658,28,All News Today; 2012/04/09; ABILA; Kronos; Sil...


In [11]:
data_json = {}
for idx in entities_data.index:
    doc_key = entities_data["doc_key"][idx]
    data_json[doc_key] = entities_data["processed_entities"][idx]

In [12]:
data_json[3]

'Facebook; users; social media site; privacy policy; advertisers; Facebook spokesman; cookies; web browser; device; Labour MP Helen Goodman; profits; advertising revenues; John Hemming; Liberal Democrat MP; Big Brother Watch; Renate Samson; chief executive; privacy campaign group; privacy settings; ad preferences tool; Amazon'

## Get prompt

### Entities Extraction

In [8]:
data_path = "/work/pi_dhruveshpate_umass_edu/project_19/ReDocREDPreprocessing/Re-DocRED/processed/"
file_v2 = "Re-DocRED_Processed_Train.csv"
    
def get_entity_example(idx, datatmp):
    allEntities = set()
    for item in datatmp["Triplets"][idx].split("\n"):
        item = item.split(" | ")
        if len(item) == 3:
            allEntities.add(item[0])
            allEntities.add(item[2])

    return datatmp["Text"][idx], '; '.join(list(allEntities))
    
def get_entities_prompt(text):
    
    data_v2 = pd.read_csv(os.path.join(data_path, file_v2), skiprows = range(1, 2000), nrows = 500)
    
    ex1, exout1 = get_entity_example(1, data_v2)
    ex2, exout2 = get_entity_example(2, data_v2)

    prompt=f'''Task: Please detect all the entities from the given input Text.
Entities could be people, organization, places, concepts, dates or any other proper nouns present in the text. \
Use the following examples as reference to understand the task. \
Give the output in the same format as given in the Example Entities Output, i.e., separated by a semicolon, ';'.

Example Text 1: {ex1}
Example Entities Output 1: {exout1}

Example Text 2: {ex2}
Example Entities Output 2: {exout2}

Text: {text}
Entities Output:'''
    
    return prompt

def get_system_msg():
    
    data_v2 = pd.read_csv(os.path.join(data_path, file_v2), skiprows = range(1, 2000), nrows = 500)
    
    ex1, exout1 = get_entity_example(1, data_v2)
    ex2, exout2 = get_entity_example(2, data_v2)

    prompt=f'''You are a helpful assistant and an expert in named entity extraction.
Task: Please detect all the entities from the given input Text.
Entities could be people, organization, places, concepts, dates or any other proper nouns present in the text. \
Use the following examples as reference to understand the task. \
Give the output in the same format as given in the Example Entities Output, i.e., separated by a semicolon, ';'.

Example Text 1: {ex1}
Example Entities Output 1: {exout1}

Example Text 2: {ex2}
Example Entities Output 2: {exout2}'''
    
    return prompt

def get_content(text):
    
    prompt=f'''Text: {text}
Entities Output:'''
    
    return prompt

In [9]:
print(get_system_msg())
print(get_content("hey"))

You are a helpful assistant and an expert in named entity extraction.
Task: Please detect all the entities from the given input Text.
Entities could be people, organization, places, concepts, dates or any other proper nouns present in the text. Use the following examples as reference to understand the task. Give the output in the same format as given in the Example Entities Output, i.e., separated by a semicolon, ';'.

Example Text 1: Udawalawe National Park lies on the boundary of Sabaragamuwa and Uva Provinces, in Sri Lanka. The national park was created to provide a sanctuary for wild animals displaced by the construction of the Udawalawe Reservoir on the Walawe River, as well as to protect the catchment of the reservoir. The reserve covers of land area and was established on 30 June 1972. Before the designation of the national park, the area was used for shifting cultivation (chena farming). The farmers were gradually removed once the national park was declared. The park is from Co

### Relations Extraction

In [12]:
data_path = "/work/pi_dhruveshpate_umass_edu/project_19/ReDocREDPreprocessing/Re-DocRED/processed/"
file_v2 = "Re-DocRED_Processed_Train.csv"
    
def get_example(idx, datatmp):
    allEntities = set()
    for item in datatmp["Triplets"][idx].split("\n"):
        item = item.split(" | ")
        # print(item, len(item))
        if len(item) == 3:
            allEntities.add(item[0])
            allEntities.add(item[2])

    allEntities = list(allEntities)
    # print(allEntities)
    # return datatmp["Text"][idx], allEntities, datatmp["Triplets"][idx]
    return datatmp["Text"][idx], "; ".join(allEntities), datatmp["Triplets"][idx]
    
def get_relations_prompt(text, entities):
    
    data_v2 = pd.read_csv(os.path.join(data_path, file_v2), skiprows = range(1, 2000), nrows = 500)
    
    ex1, exent1, exout1 = get_example(1, data_v2)
    ex2, exent2, exout2 = get_example(2, data_v2)

    prompt=f'''Task Description:
The task is to extract Relations between the Entity List for given text, in the form of triplets. \
Extract triplets from the given Text based solely on the relationships present in the text. \
Ensure that entities are chosen directly from the provided Entity List to maintain accuracy. \
Avoid duplicating triplets in the output. Use the provided Example Text and Relations Output as references \
to understand how to identify meaningful relationships between entities from Entity List. \
Pay attention to all potential relations between all the entities and include them in the output.

Example Text 1: {ex1}
Entity List of Text 1: {exent1}
Relations Output of Text 1: {exout1}

Example Text 2: {ex2}
Entity List of Text 2: {exent2}
Relations Output of Text 2: {exout2}

Text: {text}
Entity List: {entities}
Relations Output:'''
    
    return prompt

In [13]:
print(get_relations_prompt(data['0']['chunk_text'], "entities"))

Task Description:
The task is to extract Relations between the Entity List for given text, in the form of triplets. Extract triplets from the given Text based solely on the relationships present in the text. Ensure that entities are chosen directly from the provided Entity List to maintain accuracy. Avoid duplicating triplets in the output. Use the provided Example Text and Relations Output as references to understand how to identify meaningful relationships between entities from Entity List. Pay attention to all potential relations between all the entities and include them in the output.

Example Text 1: Udawalawe National Park lies on the boundary of Sabaragamuwa and Uva Provinces, in Sri Lanka. The national park was created to provide a sanctuary for wild animals displaced by the construction of the Udawalawe Reservoir on the Walawe River, as well as to protect the catchment of the reservoir. The reserve covers of land area and was established on 30 June 1972. Before the designation

In [16]:
for item in data:
    print(data['0']['chunk_text'])
    break

[{'chunk_index': 0, 'chunk_text': "In a revealing and emotional interview with Oprah Winfrey, Matthew Sandusky, the adopted son of convicted pedophile Jerry Sandusky, shared his harrowing experience of abuse on the OWN network.\n\nThe exclusive conversation, which aired on Thursday night, marked Matthew's first television appearance since the trial that led to his father's conviction in 2012.\n\nMatthew Sandusky, who was one of six children adopted by Jerry and Dottie Sandusky, recounted the years of sexual abuse he suffered from the age of eight to fifteen.\n\nHe described a chilling nightly ritual where his father would enter his bedroom and molest him, leaving him to cower in a fetal position in a desperate attempt to avoid the abuse.\n\nHis story first came to light when a secret tape of his interview with the police, conducted during his father's trial, was leaked to NBC, sending shockwaves through the community.", 'chunk_tokens': 163}, {'chunk_index': 1, 'chunk_text': "Jerry Sand

In [29]:
data['1'][0]['chunk_text']

'In the scenic Brecon Beacons of Mid Wales, a serene yet tragic scene unfolded at the Cantref Reservoir.\n\nThe emergency services, including local police, were engaged in a large-scale search and rescue operation.\n\nA young swimmer, believed to be a local from Merthyr Tydfil in South Wales, had vanished beneath the waters of the reservoir, sparking fears and a desperate search.\n\nThe Cantref Reservoir, a popular spot during the sweltering summer months, had attracted hundreds of visitors seeking respite from the heatwave that gripped the nation.\n\nThe police had been issuing stern warnings about the dangers of swimming in unmanned waters, emphasizing the risks associated with such activities during the heatwave.\n\nDespite these warnings, the young man, in his 20s, had taken a leap into the cool depths to escape the relentless heat, only to disappear.\n\nAs the search continued, the police utilized boats and helicopters to scour the miles of water surrounding the reservoir.'

In [30]:
entities_data["processed_entities"][1]

'Jaymi McCann; swimmer; emergency services; police; Cantref Reservoir; Brecon Beacons; Mid Wales; Merthyr Tydfil; South Wales; Met Office; health authorities; elderly people; Level 2 alerts'

In [ ]:
get_entities_prompt("text")

## Predictions

In [27]:
def get_prediction(text):
    
    prompt = get_entities_prompt(text)
    
    start = time.time()
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt},
        ],
        temperature = 0,
        max_tokens = 256,
    )
    time_diff = time.time() - start
    
    return response, time_diff

def get_chat_prediction(messages):
    
    start = time.time()
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature = 0,
        max_tokens = 256,
    )
    time_diff = time.time() - start
    
    return response, time_diff


def get_relation_prediction(text, entities):
    
    prompt = get_relations_prompt(text, entities)
    
    start = time.time()
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt},
        ],
        temperature=0,
    )
    time_diff = time.time() - start
    
    return response, time_diff

In [11]:
output_data = []
output_filename = "sdapr25_entities.csv"

### Entity Extraction

In [14]:
debug = True

all_keys = list(data.keys())

for i in tqdm(range(len(all_keys))):
    
    doc_key = all_keys[i]
    
    if doc_key != '5' and debug:
        continue
    
    # messages = [{"role": "system", "content": get_system_msg()}]
    
    for item in data[doc_key]:
        
        # print(messages)
    
        #
        text = item['chunk_text']
        chunk_idx = item['chunk_index']
        
        # messages.append({"role": "user", "content": get_content(text)})
        
        output, time_diff = get_prediction(text)
        # output, time_diff = get_chat_prediction(messages)
        # output, time_diff = get_relation_prediction(i)

        outputText = output.choices[0].message.content
        input_tokens = output.usage.prompt_tokens
        output_tokens = output.usage.completion_tokens
        
        # messages.append({"role": "assistant", "content": outputText})

        if debug:
            print(f"\nInput - {text}")
            # print(f"\nGT - {data['Triplets'][i]}")
            print(f"\noutput - {outputText}")

        output_data.append([doc_key, chunk_idx, outputText, time_diff,
                           input_tokens, output_tokens])

        if i % 10 == 0 and i > 1:

            with open(output_filename, 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerows(output_data)
                
        # if debug: break
        
    if debug: break
    
    with open(output_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(output_data)

  0%|          | 0/924 [00:00<?, ?it/s]


Input - In a recent development that has sparked both interest and controversy, Mothers Against Drunk Driving (MADD), a prominent anti-drink driving group, has released a glowing report on Uber, the widely-used ride-sharing service.

The report, which was jointly published with Uber, suggests that the availability of transportation options like Uber has contributed to a reduction in drink-driving accidents across America's cities.

The study has come under the spotlight after it was revealed that Uber started donating money to MADD last summer, raising questions about the potential influence of financial relationships on research outcomes.

Despite the scrutiny, MADD, backed by its senior vice president of marketing and communications, Amy George, is not backing away from the report's findings.

output - Mothers Against Drunk Driving; MADD; Uber; America; Amy George

Input - Amy George has been actively communicating with media outlets such as Pro Publica and Valley Wag, emphasizing t

  1%|          | 5/924 [00:10<31:22,  2.05s/it]


Input - With the report's findings being a topic of discussion among policymakers, researchers, and the public, the conversation about the relationship between ride-sharing services and road safety is far from over.

The ongoing dialogue underscores the importance of examining the correlations between data and real-world outcomes, as well as the need for transparency in financial relationships between organizations and their research partners.

output - ride-sharing services; road safety; policymakers; researchers; financial relationships; organizations; research partners


### Relation Extraction

In [14]:
processed = set()

In [16]:
processed

{('0', 0), ('0', 1), ('0', 2), ('0', 3), ('0', 4), ('0', 5)}

In [20]:
debug = True

all_keys = list(data.keys())

for i in tqdm(range(len(all_keys))):
    
    doc_key = all_keys[i]
    
    # if doc_key != '3' and debug:
    #     continue
        
    # if doc_key != '400':
    #     continue
    
    # messages = [{"role": "system", "content": get_system_msg()}]
    
    for item in data[doc_key]:
        
        # print(doc_key, item)
        
        # print(messages)
    
        #
        text = item['chunk_text']
        chunk_idx = item['chunk_index']
        
        # print(doc_key, chunk_idx)
        
        
        # if (doc_key, chunk_idx) != ('3',0):
        #     continue
            
        # if (doc_key, chunk_idx) in processed:
        #     continue
        # else:
        #     processed.add((doc_key, chunk_idx))
        
        # if int(chunk_index) in data_json[int(doc_key)]:
        processed_entities = data_json[int(doc_key)][chunk_idx]

        # messages.append({"role": "user", "content": get_content(text)})

        # output, time_diff = get_prediction(text)
        # output, time_diff = get_chat_prediction(messages)
        output, time_diff = get_relation_prediction(i, processed_entities)

        outputText = output.choices[0].message.content
        input_tokens = output.usage.prompt_tokens
        output_tokens = output.usage.completion_tokens

        # messages.append({"role": "assistant", "content": outputText})

        if debug:
            print(f"\nInput - {text}")
            # print(f"\nGT - {data['Triplets'][i]}")
            print(f"\noutput - {outputText}")

        output_data.append([doc_key, chunk_idx, outputText, time_diff,
                           input_tokens, output_tokens])
            
        # else:
        #     output_data.append([doc_key, chunk_idx, "No data", 0,
        #                        0, 0])

        if i % 10 == 0 and i > 1:

            with open(output_filename, 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerows(output_data)
                
        if debug: break
        
    if debug: break
    
    with open(output_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(output_data)

100%|██████████| 847/847 [00:06<00:00, 128.34it/s]


In [21]:
output_data[-1]

['400',
 0,
 'News Online Today | publication date | 2011/05/15\nABILA | country | Kronos\nProtectors of Kronos | country | Kronos\nDRUG RELATED ARRESTS ON THE RISE | publication date | 2011/05/15\nDRUG RELATED ARRESTS ON THE RISE | publisher | News Online Today\nProtectors of Kronos | located in the administrative territorial entity | Kronos\nABILA | located in the administrative territorial entity | Kronos\nNews Online Today | located in the administrative territorial entity | ABILA\nNews Online Today | country | Kronos\nDRUG RELATED ARRESTS ON THE RISE | located in the administrative territorial entity | ABILA\nDRUG RELATED ARRESTS ON THE RISE | country | Kronos',
 6.3870508670806885,
 1125,
 161]

In [19]:
data['Documents']['399']

[{'chunk_index': 0,
  'chunk_text': "SOURCE: News Online Today\n\nTITLE: KRONOS GOVERNMENT PLANS TAX BREAKS TO ENCOURAGE FOREIGN INVESTMENT \n\nPUBLISHED: 1992/12/12\n \nLOCATION: ABILA, Kronos \n\nThe Government of Kronos is currently planning some of the world's most generous tax breaks for gas investments.\n\nThe Government of Kronos is pitching the plan as a win-win situation given that the high-income countries will be struggling to discover and exploit ever-dwindling supplies of fossil-fuels and the developing Kronos needs income, infrastructure, and cheap energy.",
  'chunk_tokens': 85}]

In [48]:
int(0)

0

In [29]:
processed.remove(('3',0))

In [44]:
doc_key, chunk_index, data_json[3][0]

('3',
 1,
 'All News Today; 2012/04/09; ABILA; Kronos; Silvia Marek; Protectors of Kronos')

## Process Entities

In [ ]:
Data0 = pd.read_csv("openai_mc1_entities_Apr13.csv", header=None)
Data0 = Data0.rename(columns={0: 'doc_key', 1: 'chunk_index', 2: 'entities', 3: 'latency',
                            4: 'input_tokens',  5: 'output_tokens'})
Data0["processed_entities"] = ""
print(Data0.shape)

Data0 = Data0.iloc[9:]
Data0 = Data0.reset_index(drop=True)
print(Data0.shape)

Data0.head()

In [ ]:
import re

for i in range(Data0.shape[0]):
    
    # if i != 261: # 1, 5, 27
    #     continue
        
    ent = Data0["entities"][i]
    ent2 = ent.split("; ")
    
    ent3 = []
    for item in ent2:
        item = item.strip()
        if item and item not in ent3:
            ent3.append(item)
    ent3 = "; ".join(ent3)
    
#     if ent != ent3:

#         print(f"\n{i}")
#         print(ent)
#         print(f"\n{ent3}")
#         print("*"*20)
    
    Data0["processed_entities"][i] = ent3

In [ ]:
# Data0.head()
Data0.to_csv('openai_mc1_entities_Apr13_processed.csv', index=False)

## Combine results

In [3]:
homeFolder = "/work/pi_dhruveshpate_umass_edu/project_19/aishwarya/696DS-named-entity-extraction-and-linking-for-KG-construction/code/openai"

main_output = 'sdapr25_relations_combined.csv'

DataPath0 = os.path.join(homeFolder, "sdapr25_relations.csv")
DataPath1 = os.path.join(homeFolder, "sdapr25_relations_2.csv")
DataPath2 = os.path.join(homeFolder, "sdapr25_relations_3.csv")
DataPath01 = os.path.join(homeFolder, "sdapr25_relations_v2.csv")
DataPath11 = os.path.join(homeFolder, "sdapr25_relations_v2_2.csv")
DataPath21 = os.path.join(homeFolder, "sdapr25_relations_v2_3.csv")
# DataPath3 = os.path.join(homeFolder, "openai_relations_gt_redocred_dev_v6.csv")

Data0 = pd.read_csv(DataPath0, header=None)
Data1 = pd.read_csv(DataPath1, header=None)
Data2 = pd.read_csv(DataPath2, header=None)
Data01 = pd.read_csv(DataPath01, header=None)
Data11 = pd.read_csv(DataPath11, header=None)
Data21 = pd.read_csv(DataPath21, header=None)
# Data3 = pd.read_csv(DataPath3, header=None)
Data = pd.concat([Data0, Data1, Data2, Data01, Data11, Data21], axis=0)

Data = Data.rename(columns={0: 'doc_key', 1: 'chunk_index', 2: 'triplets', 3: 'latency',
                           4: 'input_tokens',  5: 'output_tokens', 6: 'article_name'})

Data.to_csv(main_output, index=False)

# print(Data.shape, len(list(set(list(Data["original_index"])))))
Data.head()

,doc_key,chunk_index,triplets,latency,input_tokens,output_tokens
0,0,0,Jerry Sandusky | spouse | Dottie Sandusky\nMat...,23.787573,1132,788
1,0,1,Jerry Sandusky | spouse | Dottie Sandusky\nMat...,9.572027,1132,309
2,0,2,Jerry Sandusky | spouse | Dottie Sandusky\nMat...,11.178822,1132,294
3,0,3,Jerry Sandusky | spouse | Dottie Sandusky\nMat...,18.253995,1132,557
4,0,4,Jerry Sandusky | spouse | Dottie Sandusky\nMat...,17.109828,1132,412


In [ ]:
len(list(set(list(Data["original_index"]))))

In [ ]:
check_id = list(set(list(Data["original_index"])))

In [ ]:
check_id

## Analyse output

In [ ]:
homeFolder = "/work/pi_dhruveshpate_umass_edu/project_19/aishwarya/696DS-named-entity-extraction-and-linking-for-KG-construction/code/openai"

DataPath0 = 'openai_relations_gt_redocred_dev_all.csv'

Data0 = pd.read_csv(DataPath0)

Data0.head()

In [ ]:
Data0['entities'][4]